<a href="https://colab.research.google.com/github/YunDudali/YunDudali-TAMU_CSCE636_Project_2-/blob/main/CSCE_636_Project2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **TAMU - CSCE636 Project_2**

# **Project Requirement : **

This project is on machine translation for two "artificial" languages: an "Input Language" and an "Output Language". We want to build a model to translate the texts in the "Input Language" to texts in the "Output Language". For example, a text in the "Input Language" can be "a g b f a f a e a k a j c f b f c d a k a k c e b g a h a k b d b f b f b d c d " , and its translation to the "Output Language" is "b f c f b f c d a j e f g c e b g a k i j b d b f a k l m b f b d a h ed ee ef a k k eg a k h eh a e ei c d a f ej ek a g d el".

As a training dataset, a list of 5,000 texts in the Input Language are here, and the corresponding list of 5,000 texts in the Output Language are here.  They are two lists of strings in Python.)

Your task is to design and train a good model, which can take texts in the "Input language" as its input, translate them to the corresponding texts in the "Output Language" and output them. 



# **Projetct Details Code: **

**This Block is for Testing, Please change the file path for training set,  testing set and the model** \\

I use three independent "Transformer" models is to emulating the concept of Random Forest to get the majority vote for each output token during the translation process. In this case, the translation can be more accurate and stable.

The translation process might cost some time.

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers
import pickle
import numpy as np
import random
from collections import Counter
from google.colab import drive
drive.mount('/content/drive')

# Load Model here
model = keras.models.load_model("/content/drive/MyDrive/courses/Deep Learning CSCE 636/Proj2/yundu.keras")
# Please change the training input and training output file path for the TextVectorization purpose
training_source_path = '/content/drive/MyDrive/courses/Deep Learning CSCE 636/Proj2/DS_5_train_input'
training_target_path = '/content/drive/MyDrive/courses/Deep Learning CSCE 636/Proj2/DS_5_train_output'
# Please change the testing input and testing output file path for the the prediction
testing_source_path = '/content/drive/MyDrive/courses/Deep Learning CSCE 636/Proj2/DS_5_train_input'
testing_target_path = '/content/drive/MyDrive/courses/Deep Learning CSCE 636/Proj2/DS_5_train_output'

##-----Testing Codes start here---
training_source = pickle.load(open(training_source_path, 'rb'))
training_target = pickle.load(open(training_target_path, 'rb'))
for i in range(len(training_target)):
    training_target[i] = "[start] " + training_target[i] + " [end]"
source_vectorization = layers.TextVectorization(max_tokens=20000, output_mode='int', output_sequence_length=256)
target_vectorization = layers.TextVectorization(max_tokens=20000, output_mode='int', output_sequence_length=256 + 1)
source_vectorization.adapt(training_source)
target_vectorization.adapt(training_target)

testing_source = pickle.load(open(testing_source_path, 'rb'))
testing_target = pickle.load(open(testing_target_path, 'rb'))
for i in range(len(testing_target)):
    testing_target[i] = "[start] " + testing_target[i] + " [end]"

##-----vectorization---------------
target_vocab = target_vectorization.get_vocabulary()
target_index_lookup = dict(zip(range(len(target_vocab)), target_vocab))


##-----Translate----------------
def decoded_seq(input_sentence_):
    tokenized_input = source_vectorization([input_sentence_])
    decoded_sentence = "[start]"
    for i in range(256):
        tokenized_target_sentence = target_vectorization([decoded_sentence])[:, :-1]
        [prediction1, prediction2, prediction3] = model([tokenized_input, tokenized_target_sentence])
        index_1 = np.argmax(prediction1[0, i, :])
        index_2 = np.argmax(prediction2[0, i, :])
        index_3 = np.argmax(prediction3[0, i, :])

        index_a = Counter([index_1, index_2, index_3]).most_common()[0][0]
        sampled_token = target_index_lookup[index_a]
        decoded_sentence += " " + sampled_token
        if sampled_token == "end":
            break
    return decoded_sentence


pre = []  # This is the translated sentences
num_words = 0
accu = 0
num_sentence = 10   # len(testing_source)
for idx in range(num_sentence):
    input_sentence = testing_source[idx]
    output_sentence = testing_target[idx]
    decoder_out = decoded_seq(input_sentence)

    output_sentence_ = output_sentence.replace("[start]", "").replace("[end]", "").split()
    decoder_out_ = decoder_out.replace("[start]", "").replace("end", "").split()
    pre.append(decoder_out.replace("[start]", "").replace("end", ""))

    length = min(len(output_sentence_), len(decoder_out_))
    accu += np.sum(np.array(output_sentence_[:length]) == np.array(decoder_out_[:length]))
    num_words += len(output_sentence_)

print("Translation Accuracy:")
print(accu / num_words)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Translation Accuracy:
0.7985480943738656


In [ ]:
pre

[' b f c f b f c d a j e f g c e b g a k i j b d b f a k l m b f b d a h ed ee ef a k k eg a k h eh a e ei c d a f ej ek a g d el ',
 ' b f b e c d b g c g a i f g h a f e i c e a i d j k a e l a e m c f c d a e ef c d a i ee eg eh a e ei a k ed ej ',
 ' c f b e c d b f b e a k g h b e a k i j b d c d a h k l m c d a e ee a h f ed ef a g e eg b f a d eh ei a k d ej ',
 ' b g b e c g a g e f c g a j d g h b g b f a g j k b f a k l m b e a k ed ee c e a j i ef eg c e b g a h eh ei ej ',
 ' b d c f c e a f e f c d a d g h c f c f a j i j k a e l c f a k m ed c d c d a f ef eg b g c g a g ei ej a h ee eh ek b d c e a h el em fd c e a g fe ff a k d fg ',
 ' c g b g b d c f c g a e h a f g i a d f j c e a f k l c d c e a e ee a g ed ef a d m eg b d a j e eh ei a k d ej ',
 ' c e c f b e a d e f c d c g b e c f c d a k k l a h i j m a h g h ed a e ee a g d ef c d b g a h eg eh ei ',
 ' c e c g b e a e f a e g a d e h c e a d i j a d d k a e l c f a d m ed b f c d c f a k eg eh b f a i ef ei e

**This Block is the Transformer Model**

In [ ]:
import random
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import pickle

##-----import data-----------------
source_path = './DS_5_train_input'
target_path = './DS_5_train_output'
source = pickle.load(open(source_path, 'rb'))
target = pickle.load(open(target_path, 'rb'))

temp = list(zip(source, target))
random.shuffle(temp)
source, target = zip(*temp)
source, target = list(source), list(target)

##-----Pre-process-----------------
# Add "start" and "end" tokens to every output sentence
for i in range(len(target)):
    target[i] = "[start] " + target[i] + " [end]"
# Saperate the data set into training and validation set
val_source = source[:1000]
val_target = target[:1000]
train_source = source[1000:]
train_target = target[1000:]

##-----Global Variables------------
max_tokens = 20000
max_sequence_length = 256

embed_dim = 256
num_heads = 8
dense_dim = 1024

num_encoder_block = 1
num_decoder_block = 1

##-----vectorization---------------
source_vectorization = layers.TextVectorization(max_tokens=max_tokens, output_mode='int', output_sequence_length=max_sequence_length)
target_vectorization = layers.TextVectorization(max_tokens=max_tokens, output_mode='int', output_sequence_length=max_sequence_length + 1)
source_vectorization.adapt(source)
target_vectorization.adapt(target)

##-----Data Set Creation---------
def format_dataset(src, tgt):
    src = source_vectorization(src)
    tgt = target_vectorization(tgt)
    return {"source": src, "target": tgt[:, :-1], }, tgt[:, 1:]


def make_dataset(src, tgt):
    dataset = tf.data.Dataset.from_tensor_slices((src, tgt))
    dataset = dataset.batch(32)
    dataset = dataset.map(format_dataset, num_parallel_calls=4)
    return dataset.shuffle(2048).prefetch(32).cache()


train_ds = make_dataset(train_source, train_target)
val_ds = make_dataset(val_source, val_target)


##-----Transformer model-----------
# Transformer Block
def Transformer_Encoder_Block(inputs, mask_, embed_dim_, num_heads_, dense_dim_):
    x = layers.MultiHeadAttention(num_heads=num_heads_, key_dim=embed_dim, dropout=0.2)(inputs, inputs, attention_mask=mask_[:, tf.newaxis, :])
    res = layers.LayerNormalization()(x + inputs)
    res = layers.Dropout(0.2)(res)
    x = layers.Dense(dense_dim_, activation="relu")(res)
    x = layers.Dropout(0.2)(x)
    x = layers.Dense(embed_dim_)(x)
    out = layers.LayerNormalization()(x + res)
    return out


# Positional Embedding
def Positional_Embedding(inputs):
    length = tf.shape(inputs)[-1]
    positions = tf.range(start=0, limit=length, delta=1)
    embedded_tokens = layers.Embedding(input_dim=max_tokens, output_dim=embed_dim)(inputs)
    embedded_positions = layers.Embedding(input_dim=max_sequence_length, output_dim=embed_dim)(positions)
    embedded_out = embedded_positions + embedded_tokens
    return embedded_out, tf.math.not_equal(inputs, 0)


# Decoder Mask
def Decoder_Attention_Mask(inputs):
    input_shape = tf.shape(inputs)
    batch_size, sequence_length = input_shape[0], input_shape[1]
    i = tf.range(sequence_length)[:, tf.newaxis]
    j = tf.range(sequence_length)
    mask = tf.cast(i >= j, dtype="int32")
    mask = tf.reshape(mask, (1, sequence_length, sequence_length))
    mult = tf.concat([tf.expand_dims(batch_size, -1), tf.constant([1, 1], dtype=tf.int32)], axis=0)
    return tf.tile(mask, mult)


# Transformer Decoder Block
def Transformer_Decoder_Block(inputs, encoder_outputs, mask_, embed_dim_, num_heads_, dense_dim_):
    causal_mask = Decoder_Attention_Mask(inputs)
    padding_mask = tf.cast(mask_[:, tf.newaxis, :], dtype="int32")
    padding_mask = tf.minimum(padding_mask, causal_mask)

    attention_output1 = layers.MultiHeadAttention(num_heads=num_heads_, key_dim=embed_dim_, dropout=0.2)(query=inputs, value=inputs, key=inputs, attention_mask=causal_mask)
    attention_output1 = layers.LayerNormalization()(attention_output1 + inputs)

    attention_output2 = layers.MultiHeadAttention(num_heads=num_heads_, key_dim=embed_dim_, dropout=0.2)(query=attention_output1,
                                                         value=encoder_outputs,
                                                         key=encoder_outputs,
                                                         attention_mask=padding_mask)
    attention_output2 = layers.LayerNormalization()(attention_output1 + attention_output2)
    attention_output2 = layers.Dropout(0.2)(attention_output2)
    x = layers.Dense(dense_dim_, activation="relu")(attention_output2)
    x = layers.Dropout(0.2)(x)
    x = layers.Dense(embed_dim_)(x)
    out = layers.LayerNormalization()(x + attention_output2)
    return out


# Build the whole Transformer Model
input_source = keras.Input(shape=(max_sequence_length,), dtype="int64", name="source")
input_target = keras.Input(shape=(max_sequence_length,), dtype="int64", name="target")

embedded_source, source_mask = Positional_Embedding(input_source)
embedded_target, target_mask = Positional_Embedding(input_target)

encoder_out = embedded_source
for i in range(num_encoder_block):
    encoder_out = Transformer_Encoder_Block(inputs=encoder_out, mask_=source_mask, embed_dim_=embed_dim, num_heads_=num_heads, dense_dim_=dense_dim)
decoder_out = embedded_target
for i in range(num_decoder_block):
    decoder_out = Transformer_Decoder_Block(inputs=decoder_out, encoder_outputs=encoder_out, mask_=target_mask, embed_dim_=embed_dim, num_heads_=num_heads, dense_dim_=dense_dim)
decoder_out = layers.Dense(max_tokens, activation="softmax")(decoder_out)


transformer = keras.Model([input_source, input_target], decoder_out)
transformer.summary()

transformer.compile(optimizer="rmsprop", loss="sparse_categorical_crossentropy", metrics=["accuracy"])
callbacks = [
    keras.callbacks.ModelCheckpoint(
        filepath="./project2_v2.keras",
        save_best_only=True,
        monitor="val_loss")
]

train_source, train_target = np.asarray(train_source), np.asarray(train_target)
val_source, val_target = np.asarray(val_source), np.asarray(val_target)
transformer.fit(train_ds, epochs=500,
                callbacks=callbacks,
                validation_data=val_ds, shuffle=True)


**This Block is compiling three different Transformer Models using previous block**

The purpose of using three Transformer models is to emulating the concept of Random Forest to get the majority vote for each output token. In this case, the translation can be more accurate and stable.

In [ ]:
from tensorflow import keras

transformer1 = keras.models.load_model("./project2_v1.keras", compile=False)
transformer2 = keras.models.load_model("./project2_v2.keras", compile=False)
transformer3 = keras.models.load_model("./project2_v3.keras", compile=False)

transformer1._name = "trans_1"
transformer2._name = "trans_2"
transformer3._name = "trans_3"

max_sequence_length = 256

input_source = keras.Input(shape=(max_sequence_length,), dtype="int64", name="source")
input_target = keras.Input(shape=(max_sequence_length,), dtype="int64", name="target")

# transformer1.get_layer(index=0) = input_target

output1 = transformer1([input_source, input_target])
output2 = transformer2([input_source, input_target])
output3 = transformer3([input_source, input_target])

transformer = keras.Model([input_source, input_target], [output1, output2, output3])
transformer.summary()
transformer.save("./transformer_compile2.keras")
